### SqueezeNet Introduction

[Reference1](https://blog.csdn.net/xbinworld/article/details/50897870)
[Reference2](https://zhuanlan.zhihu.com/p/49465950)

In [1]:
import warnings
warnings.filterwarnings('ignore')

import keras.backend as K

from keras.models import Model
from keras.layers import Input, Flatten, Dropout, Concatenate, Activation
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D

from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.utils.data_utils import get_file

def _fire(x, filters, name="fire"):
    sq_filters, ex1_filters, ex2_filters = filters
    squeeze = Convolution2D(sq_filters, (1, 1), activation='relu', 
                            padding='same', name=name + "/squeeze1x1")(x)
    expand1 = Convolution2D(ex1_filters, (1, 1), activation='relu', 
                            padding='same', name=name + "/expand1x1")(squeeze)
    expand2 = Convolution2D(ex2_filters, (3, 3), activation='relu', 
                            padding='same', name=name + "/expand3x3")(squeeze)
    x = Concatenate(axis=-1, name=name)([expand1, expand2])
    return x

def SqueezeNet(include_top=True, weights="imagenet", input_tensor=None, input_shape=None, 
               pooling=None, classes=1000):

    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')
    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),require_flatten=True)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    x = Convolution2D(64, kernel_size=(3, 3), strides=(2, 2), padding="same", 
                      activation="relu", name='conv1')(img_input)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool1', padding="valid")(x)

    x = _fire(x, (16, 64, 64), name="fire2")
    x = _fire(x, (16, 64, 64), name="fire3")

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool3', padding="valid")(x)

    x = _fire(x, (32, 128, 128), name="fire4")
    x = _fire(x, (32, 128, 128), name="fire5")

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool5', padding="valid")(x)

    x = _fire(x, (48, 192, 192), name="fire6")
    x = _fire(x, (48, 192, 192), name="fire7")

    x = _fire(x, (64, 256, 256), name="fire8")
    x = _fire(x, (64, 256, 256), name="fire9")

    if include_top:
        x = Dropout(0.5, name='dropout9')(x)

        x = Convolution2D(classes, (1, 1), padding='valid', name='conv10')(x)
        x = AveragePooling2D(pool_size=(13, 13), name='avgpool10')(x)
        x = Flatten(name='flatten10')(x)
#         x = Activation("softmax", name='softmax')(x)
    else:
        if pooling == "avg":
            x = GlobalAveragePooling2D(name="avgpool10")(x)
        else:
            x = GlobalMaxPooling2D(name="maxpool10")(x)

    model = Model(img_input, x, name="squeezenet")

    if weights == 'imagenet':
        model.load_weights('squeezenet_weights.h5')

    return model

Using TensorFlow backend.


In [3]:
model = SqueezeNet(weights='imagenet')

In [4]:
W = K.eval(model.weights[0])
print(W.shape)

(3, 3, 3, 64)


### Load Sample Images

In [5]:
import os
import PIL
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import tensorflow as tf
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
import pandas as pd
import glob

In [6]:
def get_image_array(img_path, model, feature_column_names):
    label = img_path.split("/")[-2]
    filename = img_path.split("/")[-1]
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    pd_tmp = pd.Series(preds[0], index=feature_column_names)
    pd_tmp['file'] = img_path
    pd_tmp['label'] = int(label)
    return pd_tmp

In [7]:
workbookDir = os.getcwd()
image_path = os.path.join(workbookDir, "..", "data",
                          "mnist", "trainingSample", "*", "*")
image_files = [f for f in glob.glob(image_path) if f.endswith('.jpg')]
feature_column_names = ["feature_" + str(i+1).zfill(4) for i in range(1000)]

In [8]:
df = pd.concat([get_image_array(img, model, feature_column_names)
                for img in image_files], axis=1).T
df.head(3)

,feature_0001,feature_0002,feature_0003,feature_0004,feature_0005,feature_0006,feature_0007,feature_0008,feature_0009,feature_0010,...,feature_0993,feature_0994,feature_0995,feature_0996,feature_0997,feature_0998,feature_0999,feature_1000,file,label
0,-0.915711,-0.431413,2.68666,-1.79038,2.29478,4.04116,0.143388,6.1491,3.91765,4.81514,...,0.365376,-4.0591,-1.97262,-1.60569,-0.210801,-0.41385,2.43721,7.73984,/home/pengtan/Projects/python/image-classifica...,0
1,-0.977896,-1.14779,3.00992,-0.796752,1.97921,6.00256,0.781126,6.64005,4.33569,4.79362,...,-0.658314,-4.06554,-2.68657,-2.25821,-0.40911,-0.531806,2.79297,7.3321,/home/pengtan/Projects/python/image-classifica...,0
2,-0.615356,-2.69798,2.65719,0.269202,2.89391,5.64014,-0.482341,5.64066,3.00114,3.6852,...,0.341816,-5.61026,-2.7048,-2.40005,-1.06077,-0.799596,5.00047,8.53104,/home/pengtan/Projects/python/image-classifica...,0


In [10]:
from PIL import Image
import IPython.display
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# from skimage.measure import block_reduce
from sklearn.linear_model import LogisticRegression

In [12]:
X = df.filter(regex='feature_').values
y = df['label'].astype("str")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

SVC_model = SVC()
KNN_model = KNeighborsClassifier(n_neighbors=3)
RF_model = RandomForestClassifier(max_depth=10, random_state=0)

SVC_model.fit(X_train, y_train)
KNN_model.fit(X_train, y_train)
RF_model.fit(X_train, y_train)
clf = LogisticRegression(random_state=0).fit(X, y)

SVC_prediction = SVC_model.predict(X_test)
KNN_prediction = KNN_model.predict(X_test)
RF_prediction = RF_model.predict(X_test)
logit_prediction = clf.predict(X_test)

print(accuracy_score(SVC_prediction, y_test))
print(accuracy_score(KNN_prediction, y_test))
print(accuracy_score(RF_prediction, y_test))
print(accuracy_score(logit_prediction, y_test))

print(classification_report(SVC_prediction, y_test))
print(classification_report(KNN_prediction, y_test))
print(classification_report(RF_prediction, y_test))
print(classification_report(logit_prediction, y_test))

0.5583333333333333
0.6083333333333333
0.6333333333333333
0.975
              precision    recall  f1-score   support

           0       0.88      0.93      0.90        15
           1       0.89      0.89      0.89         9
           2       0.08      0.33      0.13         3
           3       0.00      0.00      0.00         0
           4       0.82      0.56      0.67        16
           5       0.62      0.22      0.32        23
           6       0.88      0.50      0.64        14
           7       0.53      1.00      0.69         9
           8       0.43      0.55      0.48        11
           9       0.80      0.40      0.53        20

    accuracy                           0.56       120
   macro avg       0.59      0.54      0.53       120
weighted avg       0.72      0.56      0.59       120

              precision    recall  f1-score   support

           0       0.94      0.94      0.94        16
           1       1.00      0.69      0.82        13
           2   